In [1]:
from MovieLens import MovieLens
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter
        
test_subject = '793'
k = 10

In [2]:
# Load our data set and compute the user similarity matrix
ml = MovieLens()
data = ml.load_movies_dataset()


train_set = data.build_full_trainset()

#We use cosine similarity and user_based set to False (item_based is True)
sim_options = {'name': 'cosine',
               'user_based': False
               }

In [3]:
#Build the model using KNN basic with k =40 by defult. There are other algorithms such as 
#KNNWithMeans, KNNWithZScore, KNNBaseline
model = KNNBasic(sim_options=sim_options)

#Fit the model on the train_set
model.fit(train_set)

#Compute the similarity matrix
sims_matrix = model.compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [4]:
test_user_inner_ID = train_set.to_inner_uid(test_subject)

# Get the top K items we rated
test_user_ratings = train_set.ur[test_user_inner_ID]

In [5]:
k_neighbors = []
for rating in test_user_ratings:
    if rating[1] > 4.0:
        k_neighbors.append(rating)

# Get similar items to stuff we liked (weighted by rating)
candidates = defaultdict(float)
for itemID, rating in k_neighbors:
    similarity_row = sims_matrix[itemID]
    for innerID, score in enumerate(similarity_row):
        candidates[innerID] += score * (rating / 5.0)
    
# Build a dictionary of stuff the user has already seen
watched = {}
for itemID, rating in train_set.ur[test_user_inner_ID]:
    watched[itemID] = 1

In [6]:
# Get top-rated items from similar users:
pos = 0
for itemID, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        movieID = train_set.to_raw_iid(itemID)
        print(ml.get_movie_name(int(movieID)), rating_sum)
        pos += 1
        if (pos > 10):
            break

Making Mr. Right (1987) 3.0
Sunshine State (2002) 3.0
Program, The (1993) 3.0
Fifty Shades of Grey (2015) 3.0
Surf Nazis Must Die (1987) 3.0
Fellini Satyricon (1969) 3.0
Santa Sangre (1989) 3.0
Lola MontÃ¨s (1955) 3.0
Dresser, The (1983) 3.0
Dog Days (Hundstage) (2001) 3.0
Tanguy (2001) 3.0
